In [9]:
import random
import numpy as np
import math


class LinkSearch():
    def link(self, x1, y1, x2, y2):
        self.map[x1, y1] = self.map[x2, y2] = 0

    def direct(self, x1, y1, x2, y2):
        if x1 == x2 and y1 == y2:
            return False
        elif x1 == x2:    # 平行y轴
            return not self.map[x1, y1:y2:(2*(y1 < y2)-1)][1:].any()
        elif y1 == y2:    # 平行x轴
            return not self.map[x1:x2:(2*(x1 < x2)-1), y1][1:].any()
        return False

    def is_empty(self, x, y):
        return not self.map[x, y]

    def get_boarder(self, x, y, direction):
        if direction=='x':
            a = b = x
            while a>0 and self.is_empty(a-1, y):
                a -= 1
            while b<9 and self.is_empty(b+1, y):
                b += 1
        elif direction=='y':
            a = b = y
            while a>0 and self.is_empty(x, a-1):
                a -= 1
            while b<9 and self.is_empty(x, b+1):
                b += 1
        return a, b + 1

    def one_corner(self, x1, y1, x2, y2):
        p1 = self.direct(x1, y2, x1, y1) and self.direct(x1, y2, x2, y2) and not self.map[x1, y2] # x1, y2
        p2 = self.direct(x2, y1, x1, y1) and self.direct(x2, y1, x2, y2) and not self.map[x2, y1] # x2, y1
        return p1 or p2
        
    def two_corner(self, x1, y1, x2, y2):
        a1, b1 = self.get_boarder(x1, y1, 'y')
        a2, b2 = self.get_boarder(x2, y2, 'y')
        for y in range(max(a1, a2), min(b1, b2)):
            if self.direct(x1, y, x2, y):
                return True
        
        a1, b1 = self.get_boarder(x1, y1, 'x')
        a2, b2 = self.get_boarder(x2, y2, 'x')
        for x in range(max(a1, a2), min(b1, b2)):
            if self.direct(x, y1, x, y2):
                return True

        return False

    def three_corner(self, x1, y1, x2, y2):
        ax1, bx1 = self.get_boarder(x1, y1, 'x')
        ay2, by2 = self.get_boarder(x2, y2, 'y')
        ax2, bx2 = self.get_boarder(x2, y2, 'x')
        ay1, by1 = self.get_boarder(x1, y1, 'y')
        
        for x in range(ax1, bx1):
            for y in range(ay2, by2):
                if self.is_empty(x, y) and self.direct(x2, y, x, y) and self.direct(x, y, x, y1):
                    return True

        for x in range(ax2, bx2):
            for y in range(ay1, by1):
                if self.is_empty(x, y) and self.direct(x1, y, x, y) and self.direct(x, y, x, y2):
                    return True
        
        return False

    def search_link(self, x1, y1, x2, y2):
        if x1 == x2 and y1 == y2:
            return None
        if self.direct(x1, y1, x2, y2):
            # print('1 line.') 50
            return 50
        elif self.one_corner(x1, y1, x2, y2):
            # print('2 lines.') 20
            return 20
        elif self.two_corner(x1, y1, x2, y2):
            # print('3 lines.') 10
            return 10
        elif self.three_corner(x1, y1, x2, y2):
            # print('4 lines.') 0
            return 0
        else:
            # print('more than 4 lines.')
            return None
    
    def is_all_empty(self):
        return not self.map.any()

ls = LinkSearch()

def generate_map():
    sample_list = random.choices(range(1, 30), k=32) * 2
    return np.array(sample_list, dtype=np.int).reshape(8, 8)

map_orig = np.pad(generate_map(), ((1,1),(1,1)), 'constant', constant_values=(0,0))

rand_list = [(x, y) for x in range(1, 10) for y in range(1, 10)]
def create_new_path(rand_list):
    ls.map = map_orig.copy()
    rand_range_len = random.randrange(5, 15)
    a = random.randrange(0, 64 - rand_range_len)
    b = a + rand_range_len
    rand_list[a:b] = random.sample(rand_list[a:b], k=rand_range_len)

    score = 0
    path = []

    for x, y in rand_list:
        num = ls.map[x, y]
        if num == 0:
            continue

        sl, pl = [], []
        for p in np.argwhere(ls.map == num):
            s = ls.search_link(x, y, *p)
            if s is not None:
                sl.append(s)
                pl.append(p)

        if len(sl):
            s, p = random.choice(list(zip(sl, pl)))
            ls.link(x, y, *p)
            path.append([x, y, p[0], p[1]])
            score += s
    
    if ls.is_all_empty():
        return score, path, rand_list
    else:
        return create_new_path(rand_list)

# for _ in range(20):
#     # create_new_path()
#     print(create_new_path(rand_list.copy()))
#     pass

In [10]:
%%timeit
create_new_path(rand_list.copy())

15.7 ms ± 2.89 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
score_list = []
def SA():
    # notes
    # q = 0.98, tb = 6000, te = T_begin * (q ** 300), m_l = 1000
    # q = 0.99, tb = 3000, te = T_begin * (q ** 400), m_l = 1000

    # Cool down speed
    q = 0.99

    T_begin = 3000.0
    T_end = T_begin * (q ** 400)
    T = T_begin

    # Iteration per Temp
    mapkob_len = 10

    # init path
    rand_list = [(x, y) for x in range(1, 10) for y in range(1, 10)]
    score = 0


    while T > T_end:
        for _ in range(mapkob_len):

            new_score, new_path, new_rand_list = create_new_path(rand_list.copy())

            df = score - new_score

            if df < 0:
                path = new_path.copy()
                score = new_score
                rand_list = new_rand_list.copy()

            elif math.exp(-df/T) > random.random():
                path = new_path.copy()
                score = new_score
                rand_list = new_rand_list.copy()
            
            print(score)

        T *= q

    return score, path


In [4]:
SA()

590
540
580
740
670
770
650
750
530
720
720
680
800
520
780
590
720
680
610
640
620
730
730
680
770
650
680
720
690
730
670
750
600
670
670
700
680
650
820
660
700
640
770
660
630
720
760
750
690
640
600
690
730
750
760
760
760
800
750
690
700
520
680
680
650
780
680
750
910
640
730
670
650
670
740
790
740
720
690
790
730
750
820
720
810
780
810
790
700
750
670
670
610
610
740
670
600
730
790
680
750
510
760
760
730
620
680
720
710
760
660
700
780
680
650
760
670
690
760
770
770
770
750
700
820
770
700
770
740
800
740
770
820
700
680
820
740
810
810
780
780
840
770
720
670
760
800
800
780
780
770
790
690
620
710
780
730
690
680
810
760
750
640
680
920
760
660
630
780
580
800
670
780
710
820
770
640
720
820
730
860
700
740
840
660
770
810
800
740
820
750
800
750
790
660
630
720
680
680
720
800
650
770
700
580
740
660
780
750
750
630
640
690
740
720
600
740
760
690
840
880
630
710
680
670
690
820
770
740
700
780
790
700
740
720
670
850
710
790
700
810
720
570
850
880
700
770
760
840
730


KeyboardInterrupt: 